# Results exporter
Notebook to easily load a sacred experiment and create a CSV file ready to be used in Latex.

In [1]:
import sys
sys.path = ['..'] + sys.path
import pandas as pd
import numpy as np
import glob, sys, json, re
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sts
from sacred_notebook_utils import plot_mean_ci, plot_all
from baselines.common.sacred_utils import load_runs, filter_runs
import scipy.stats as sts

In [2]:
runs = load_runs('../../../sacred_data/sacred_runs_master/')
print("Loaded runs:", len(runs.keys()))
final_runs = load_runs('../../../sacred_data/sacred_runs_final/')
print("Loaded final runs:", len(final_runs.keys()))

Loaded runs: 572
Loaded final runs: 704


In [3]:
pdisnn_runs_local = load_runs('../sacred_pdisnn/')
print("Loaded pdisnn runs:", len(pdisnn_runs_local.keys()))
# Merge the runs
total_runs = {}
for key, value in runs.items():
    total_runs[str(key) + '-base'] = value
for key, value in final_runs.items():
    total_runs[str(key) + '-final'] = value
for key, value in pdisnn_runs_local.items():
    total_runs[str(key) + '-local'] = value

Loaded pdisnn runs: 293


## Load the experiment we need

In [61]:
query = {
    'run.status': 'COMPLETED',
    'config.center': False,
    'config.gamma': 1.0,
    'config.clipping': False,
    'config.natural': False,
    'config.policy': 'linear',
    'config.iw_norm': 'none',
    'config.entropy': 'none',
    'config.env': 'rllab.cartpole',
    'config.iw_method': 'pdis',
    'config.delta': 0.99,
    'config.bound': 'pdis-max-d2',
    'config.num_episodes': 20,
    'config.experiment_name': 'pdis_variance_test_v2std'
}
filtered_runs = filter_runs(query, total_runs)
print("Experiments:", len(filtered_runs.keys()))
print("Seeds:", ", ".join(map(str, [filtered_runs[key]['config']['seed'] for key in filtered_runs.keys()])))

Experiments: 20
Seeds: 315, 689, 307, 947, 15, 963, 712, 662, 746, 100, 245, 42, 8, 247, 545, 178, 236, 481, 950, 153


## Extract the data we need for the CSV

In [62]:
runs = list(filtered_runs.keys())
df = []
for run_key in runs:
    episodes = filtered_runs[run_key]['metrics']['Iteration']['values']
    #rewards = filtered_runs[run_key]['metrics']['EpRewMean']['values']
    stdiw_pdis = filtered_runs[run_key]['metrics']['J_sample_variance']['values']
    stdiw_is = filtered_runs[run_key]['metrics']['IS_J_sample_variance']['values']
    ratio = (np.array(stdiw_is) - np.array(stdiw_pdis)) / np.abs(np.array(stdiw_pdis))
    df.append(pd.Series(ratio, episodes))
performance = pd.concat(df, axis=1)
values = performance.values
initial_columns = list(performance)
performance['mean'] = np.mean(values, axis=1)
performance['std'] = np.std(values, axis=1)
low, up = sts.t.interval(0.95, values.shape[1]-1, loc=np.mean(values, axis=1), scale=sts.sem(values, axis=1))
performance['low'] = low
performance['up'] = up
performance.drop(columns=initial_columns, inplace=True)
performance.reset_index(inplace=True)
performance.rename(columns={'index': 'iteration'}, inplace=True)
#performance['iteration'] -= 100
performance = performance.iloc[::5]
print(performance.shape)
performance.head()

(100, 5)


,iteration,mean,std,low,up
0,1,3.627119,6.063596,0.715546,6.538692
5,6,10.816402,20.409389,1.016371,20.616432
10,11,6.803235,9.618417,2.184734,11.421736
15,16,14.523561,32.872959,-1.261135,30.308257
20,21,2.148643,4.179090,0.141958,4.155328


## Create the CSV

In [63]:
performance.to_csv('variance_cartpole_jratio_20.csv', index=False)